# Análise Geoespacial

Nesta seção, abordaremos algumas análises geoespaciais super básicas com Python. Como temos coordenadas de latitude e longitude em alguns dos nossos dados de colisão com pássaros, pode fazer sentido plotá-los para ver se conseguimos identificar algumas tendências geográficas. É verdade que uma análise completa exigiria a inclusão de horários de voos e outros dados externos, mas aprenderemos apenas o suficiente para nos familiarizarmos com o assunto.

## Mapa Básico Usando Geopandas

**Análise geoespacial** usa SIG e camadas de mapa para dar sentido a padrões geográficos em dados. Trabalha com mapas geográficos. Existe uma biblioteca útil chamada Geopandas para lidar com análise geoespacial, e ela estende o Pandas com um recurso simples e inteligente: adiciona uma coluna `geometry` a um determinado dataframe.

Obter dados de mapeamento pode ser bastante complexo, mas existe um site [Natural Earth](https://www.naturalearthdata.com/) que fornece alguns conjuntos de dados de mapeamento gratuitos para trabalhar. Já baixei alguns desses conjuntos de dados junto com este notebook. Vamos trazer o arquivo "countries" e exibi-lo.

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Baixe o shapefile mundial do site da Natural Earth
world = gpd.read_file(r"https://github.com/thomasnield/anaconda_python_eda/raw/public/resource/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp")

# Plot o mapa mundial
ax = world.plot()

# Mostre o gráfico
plt.show()

Isso nos dá um mapa-múndi. Muito bom. Mas o que exatamente é esse objeto que lemos de um arquivo em `geopandas`?

In [ ]:
world

Se isso se parece com um `DataFrame`, é porque é. Mais corretamente, é um `GeoDataFrame` e o que o diferencia é que ele possui uma coluna `geometry` para lidar com a representação de mapeamento de cada registro.

Existem algumas funções Pandas familiares e úteis que podemos usar com um `GeoDataFrame`. Por exemplo, só consigo obter registros que sejam soberanos dos Estados Unidos da América.

In [ ]:
world[world["SOVEREIGNT"] == "United States of America"]

Interessante. Obtivemos os próprios Estados Unidos, bem como seu território, Porto Rico. Também podemos simplesmente plotar esses resultados.

In [ ]:
usa = world[(world["SOVEREIGNT"] == "United States of America")]

# Plot o mapa do mundo
ax = usa.plot()

Vamos recuar e trazer não apenas o mundo inteiro, mas também todos os dados de colisão de pássaros para os quais temos dados de latitude e longitude. Lembre-se de que não temos valores de latitude e longitude para todos os registros. Vamos obter essa contagem.

In [ ]:
import pandas as pd 

df = pd.read_csv(r"https://github.com/thomasnield/anaconda_python_eda/raw/public/birdstrike_section2.csv")

((df["LATITUDE"].notna()) & df["LONGITUDE"].notna()).value_counts()

Temos 121.359 registros com valores de latitude e longitude e 19.710 registros sem valores. Podemos conviver com isso e explorar por que esses locais não foram fornecidos. Mas deixaremos nossos gráficos ignorarem esses valores ausentes.

Vamos agora criar um `GeoDataFrame` separado, chamado `gdf`, que receberá os dados de colisão com pássaros e, em seguida, adicionará a coluna `geometry` usando as colunas `LONGITUDE` e `LATITUDE`. O GeoPandas possui uma função conveniente, `points_from_xy`, que fará essa conversão.

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Baixe o shapefile mundial do site da Natural Earth
world = gpd.read_file(r"https://github.com/thomasnield/anaconda_python_eda/raw/public/resource/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp")

gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(x=df["LONGITUDE"],y=df["LATITUDE"]),crs="EPSG:4326")

# Plot o mapa mundial
ax = world.plot()

gdf.plot(ax=ax, color='red', markersize=5)

# Mostre o gráfico
plt.show()

E, só por curiosidade, vamos dar uma olhada nesse `GeoDataFrame`. Com certeza, ele possui a coluna `geometry`, que permite que o gráfico acima seja possível.

In [ ]:
with pd.option_context('display.max_columns', None):
    display(gdf)

Podemos "dar um zoom" apenas nos Estados Unidos, se quisermos, já que esses dados vêm de uma agência americana: a FAA. Isso significa que veremos um forte viés de incidentes nos EUA, já que a FAA rastreia apenas voos que chegam ou partem dos EUA. Podemos usar uma junção espacial `sjoin()` para incluir apenas colisões com pássaros que existam dentro dos contornos dos Estados Unidos.

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd 

usa = world[(world["SOVEREIGNT"] == "United States of America") & (world["TYPE"] == "Country")]

# Plot o mapa mundial
ax = usa.plot()

# Plot apenas os pontos dos EUA
usa_points = gpd.sjoin(gdf, usa, predicate='within')
usa_points.plot(ax=ax, color='red', markersize=.1)

# Mostre o gráfico
plt.show()

## Províncias e Estados

Digamos que queremos ampliar ainda mais. Vamos trazer o conjunto de dados de estados e províncias.

In [ ]:
states = gpd.read_file(r"https://github.com/thomasnield/anaconda_python_eda/raw/public/resource/ne_110m_admin_1_states_provinces/ne_110m_admin_1_states_provinces.shp")
states

Observe que, quando plotamos os estados, agora vemos as fronteiras.

In [ ]:
states.plot()

Vamos remover o Alasca e o Havaí e fazer uma junção espacial apenas com colisões com pássaros no continente.

In [ ]:
mainland = states[~states['name'].isin(['Alaska', 'Hawaii'])]

mainland_points = gpd.sjoin(gdf, mainland, predicate='within')

ax = mainland.plot()
mainland_points.plot(ax=ax, color='red', markersize=.1)

plt.show()

Para a próxima parte, quero também ver os aeroportos e verificar se colisões com pássaros (sem surpresa) acontecem perto deles. Lembre-se de que colisões com pássaros são mais propensas a acontecer durante as fases de decolagem e pouso. Vamos trazer estes dados de aeroporto que obtive da FAA.

In [ ]:
airports = pd.read_csv(r"https://github.com/thomasnield/anaconda_python_eda/raw/public/resource/Airports.csv", usecols=['Icao_Ident','the_geom'])
airports 

Eu só extraí duas colunas e observei que uma delas tem uma coluna chamada `the_geom`, que achamos que funcionaria com o GeoPandas. Por algum motivo, não funcionou. Então, farei uma limpeza separando as colunas `Longitude` e `Latitude` com expressões regulares, removendo os elementos padrão, como os parênteses e `Point`, e então construirei um novo GeoDataFrame em torno desses dois valores.

In [ ]:
airports[['Longitude', 'Latitude']] = airports['the_geom'].str.split(r"(?<=[0-9])[ ](?=[-0-9])",regex=True, expand=True)

airports['Longitude'] = airports['Longitude'].str.replace('[^-0-9.]','', regex=True)
airports['Latitude'] = airports['Latitude'].str.replace('[^-0-9.]','', regex=True)

airports = gpd.GeoDataFrame(airports, geometry=gpd.points_from_xy(x=airports["Longitude"],y=airports["Latitude"]),crs="EPSG:4326")
airports

Agora posso dar uma olhada nas colisões com pássaros ao lado dos aeroportos. Vou fazer os aeroportos com pontos brancos e, para focar, vou olhar para o estado do Texas.

In [ ]:
texas = states[states['name'].isin(['Texas'])]

texas_points = gpd.sjoin(gdf, texas, predicate='within')
texas_airports = gpd.sjoin(airports, texas, predicate='within')

ax = texas.plot()
texas_airports.plot(ax=ax, color='white', markersize=1)
texas_points.plot(ax=ax, color='red', markersize=2)

plt.show()

Parece que há muitas colisões com pássaros onde há um conjunto de aeroportos. Gostaria de saber se esse conjunto no norte do Texas tem muitas colisões com pássaros para voos relacionados ao Dallas. Dallas é notoriamente movimentada em nível global. Você pode investigar isso?

Vamos olhar para a Flórida também.

In [ ]:
florida = states[states['name'].isin(['Florida'])]

florida_points = gpd.sjoin(gdf, florida, predicate='within')
florida_airports = gpd.sjoin(airports, florida, predicate='within')

ax = florida.plot()
florida_airports.plot(ax=ax, color='white', markersize=1)
florida_points.plot(ax=ax, color='red', markersize=2)

plt.show()

Há um sinal menos óbvio na Flórida. Poderia ajudar se filtrássemos aeroportos que são pistas pequenas ou minúsculas, e poderíamos filtrar isso pelo número de voos por dia. Há muitas análises interessantes que poderíamos fazer aqui, especialmente à medida que trazemos mais dados externos! Mas, por uma questão de tempo, vamos encerrar por aqui. Recomendo que você busque esses dados e continue explorando!

## Exercício

Complete o código (substituindo os pontos de interrogação "?") abaixo para plotar colisões com pássaros e aeroportos no estado de Nova York.

In [ ]:
import geopandas as gpd
import pandas as pd 
import matplotlib.pyplot as plt

df = pd.read_csv(r"https://github.com/thomasnield/anaconda_python_eda/raw/public/birdstrike_section2.csv")
gdf = gpd.GeoDataFrame(?, geometry=gpd.points_from_xy(x=df["LONGITUDE"],y=df["LATITUDE"]),crs="EPSG:4326")
states = gpd.read_file(r"https://github.com/thomasnield/anaconda_python_eda/raw/public/resource/ne_110m_admin_1_states_provinces/ne_110m_admin_1_states_provinces.shp")

airports = pd.read_csv(r"https://github.com/thomasnield/anaconda_python_eda/raw/public/resource/Airports.csv", usecols=['Icao_Ident','the_geom'])
airports[['Longitude', 'Latitude']] = airports['the_geom'].str.split(r"(?<=[0-9])[ ](?=[-0-9])",regex=True, expand=True)
airports['Longitude'] = airports['Longitude'].str.replace('[^-0-9.]','', regex=True)
airports['Latitude'] = airports['Latitude'].str.replace('[^-0-9.]','', regex=True)
airports = gpd.GeoDataFrame(?, geometry=gpd.points_from_xy(x=airports["Longitude"],y=airports["Latitude"]),crs="EPSG:4326")

new_york = states[states['name'].isin(['New York'])]

new_york_points = gpd.sjoin(?, ?, predicate=?)
new_york_airports = gpd.sjoin(?, ?, predicate=?)

ax = new_york.plot()
new_york_airports.plot(ax=ax, color='white', markersize=1)
new_york_points.plot(ax=ax, color='red', markersize=2)

plt.show()

### RESPOSTA A BAIXO

|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

In [ ]:
import geopandas as gpd
import pandas as pd 
import matplotlib.pyplot as plt

df = pd.read_csv(r"https://github.com/thomasnield/anaconda_python_eda/raw/public/birdstrike_section2.csv")
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(x=df["LONGITUDE"],y=df["LATITUDE"]),crs="EPSG:4326")
states = gpd.read_file(r"resource/ne_110m_admin_1_states_provinces/ne_110m_admin_1_states_provinces.shp")

airports = pd.read_csv(r"https://github.com/thomasnield/anaconda_python_eda/raw/public/resource/Airports.csv", usecols=['Icao_Ident','the_geom'])
airports[['Longitude', 'Latitude']] = airports['the_geom'].str.split(r"(?<=[0-9])[ ](?=[-0-9])",regex=True, expand=True)
airports['Longitude'] = airports['Longitude'].str.replace('[^-0-9.]','', regex=True)
airports['Latitude'] = airports['Latitude'].str.replace('[^-0-9.]','', regex=True)
airports = gpd.GeoDataFrame(airports, geometry=gpd.points_from_xy(x=airports["Longitude"],y=airports["Latitude"]),crs="EPSG:4326")

new_york = states[states['name'].isin(['New York'])]

new_york_points = gpd.sjoin(gdf, new_york, predicate='within')
new_york_airports = gpd.sjoin(airports, new_york, predicate='within')

ax = new_york.plot()
new_york_airports.plot(ax=ax, color='white', markersize=1)
new_york_points.plot(ax=ax, color='red', markersize=2)

plt.show()